In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
driver = webdriver.Chrome('c:/chromedriver/chromedriver.exe')
url = 'http://fifaonline4.nexon.com/datacenter/rank?n4seasonno=36'
driver.get(url)

In [11]:
def go_nextpage_10():
    nextpage_10 = driver.find_element_by_css_selector('#middle > div > div > div.board_list.datacenter_rank > div.pagination > div > a.btn_next_list.ajaxNav')
    nextpage_10.click()
    time.sleep(1)

def go_nextpage(i):
    nextpage = driver.find_element_by_css_selector('#middle > div > div > div.board_list.datacenter_rank > div.pagination > div > ul > li:nth-child('+str(i)+') > a')
    nextpage.click()
    time.sleep(1)

def get_ranker():
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    number=[]; name=[]
    
    for i in list(range(1,21)):
        tmp = soup.select('#middle > div > div > div.board_list.datacenter_rank > div.content.datacenter_rank_list > div > div.tbody > div:nth-child('+str(i)+') > span.td.rank_coach > span.coach_wrap > span.name.profile_pointer')
        data = str(tmp[0])
        number.append(data[data.index('n="')+3 : data.index('">')])
        name.append(data[data.index('">')+2 : data.index('</')])
    return number, name

In [13]:
ranker_number = []
ranker_name = []

#1위~200위
number, name = get_ranker()
ranker_number = ranker_number + number
ranker_name = ranker_name + name

for i in list(range(2,11)):
    go_nextpage(i)
    number, name = get_ranker()
    ranker_number = ranker_number + number
    ranker_name = ranker_name + name

#201위~10000위
for i in range(49):
    go_nextpage_10()
    number, name = get_ranker()
    ranker_number = ranker_number + number
    ranker_name = ranker_name + name

    for i in list(range(2,11)):
        go_nextpage(i)
        number, name = get_ranker()
        ranker_number = ranker_number + number
        ranker_name = ranker_name + name

In [14]:
ranker = {'순위':list(range(1,10001)),'이름':ranker_name,'계정번호':ranker_number}
ranker = pd.DataFrame(ranker)
ranker

,이름,계정번호
순위,,
1,WhizFox김유민,1222944439
2,대학생아닙니다,266802834
3,GalaxyXG윤창근,1256577701
4,PainT박지민,988037554
5,SaddlerSeongMin,1927285294
...,...,...
9996,분위기쐐신,1139086293
9997,김길령,434176910
9998,스톤샤워,1692793046


In [15]:
ranker[ranker.duplicated()==True]

,이름,계정번호
순위,,


In [17]:
ranker.to_csv('ranker_202104.txt')